# Spectral Analysis of Small Graphs

This notebook demonstrates how to:
1. Load graphs from graph6 encoding into NetworkX
2. Compute adjacency and Laplacian matrices
3. Calculate eigenvalues
4. Visualize spectra

No API required - this is pure local computation using NetworkX and NumPy.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# For nicer plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Loading graphs from graph6 encoding

Graph6 is a compact ASCII encoding for undirected graphs, used by tools like `nauty` and `geng`.

In [ ]:
def graph6_to_nx(g6: str) -> nx.Graph:
    """Convert graph6 string to NetworkX graph."""
    return nx.from_graph6_bytes(g6.encode())

# Some example graphs
examples = {
    'K5': 'D~{',        # Complete graph on 5 vertices
    'C6': 'E?Bw',       # 6-cycle
    'P5': 'DQo',        # Path on 5 vertices
    'K33': 'EFz_',      # Complete bipartite K_{3,3}
    'Petersen': 'IsP@PGXD_',  # Petersen graph
}

# Load all examples
graphs = {name: graph6_to_nx(g6) for name, g6 in examples.items()}

for name, G in graphs.items():
    print(f"{name}: {G.number_of_nodes()} vertices, {G.number_of_edges()} edges")

## Computing Matrix Spectra

We'll compute eigenvalues for two fundamental matrices:
- **Adjacency matrix** $A$: $A_{ij} = 1$ if vertices $i$ and $j$ are connected
- **Laplacian matrix** $L = D - A$: where $D$ is the diagonal degree matrix

In [ ]:
def compute_spectra(G: nx.Graph) -> dict:
    """Compute adjacency and Laplacian eigenvalues."""
    # Adjacency spectrum
    adj_eigs = np.sort(nx.adjacency_spectrum(G).real)
    
    # Laplacian spectrum
    lap_eigs = np.sort(nx.laplacian_spectrum(G).real)
    
    return {
        'adjacency': adj_eigs,
        'laplacian': lap_eigs
    }

# Compute spectra for all graphs
spectra = {name: compute_spectra(G) for name, G in graphs.items()}

# Display K5 spectra
print("K5 (complete graph) spectra:")
print(f"  Adjacency: {np.round(spectra['K5']['adjacency'], 4)}")
print(f"  Laplacian: {np.round(spectra['K5']['laplacian'], 4)}")

## Visualizing Spectra

Let's plot the eigenvalues on the real line to compare different graphs.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

colors = plt.cm.tab10(np.linspace(0, 1, len(graphs)))

# Adjacency spectra
ax = axes[0]
for i, (name, spec) in enumerate(spectra.items()):
    eigs = spec['adjacency']
    ax.scatter(eigs, [i] * len(eigs), c=[colors[i]], s=100, label=name, alpha=0.7)
ax.set_yticks(range(len(graphs)))
ax.set_yticklabels(list(graphs.keys()))
ax.set_xlabel('Eigenvalue')
ax.set_title('Adjacency Matrix Spectra')
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

# Laplacian spectra
ax = axes[1]
for i, (name, spec) in enumerate(spectra.items()):
    eigs = spec['laplacian']
    ax.scatter(eigs, [i] * len(eigs), c=[colors[i]], s=100, label=name, alpha=0.7)
ax.set_yticks(range(len(graphs)))
ax.set_yticklabels(list(graphs.keys()))
ax.set_xlabel('Eigenvalue')
ax.set_title('Laplacian Matrix Spectra')
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Key Spectral Properties

Let's extract some important spectral invariants:
- **Spectral radius** $\rho(G)$: largest eigenvalue of adjacency matrix
- **Algebraic connectivity** $a(G)$: second-smallest Laplacian eigenvalue (Fiedler value)
- **Energy**: sum of absolute values of adjacency eigenvalues

In [ ]:
def spectral_properties(G: nx.Graph) -> dict:
    """Compute key spectral invariants."""
    adj_eigs = np.sort(nx.adjacency_spectrum(G).real)
    lap_eigs = np.sort(nx.laplacian_spectrum(G).real)
    
    return {
        'spectral_radius': np.max(np.abs(adj_eigs)),
        'algebraic_connectivity': lap_eigs[1] if len(lap_eigs) > 1 else 0,
        'energy': np.sum(np.abs(adj_eigs)),
        'laplacian_spectral_gap': lap_eigs[-1] - lap_eigs[-2] if len(lap_eigs) > 1 else 0
    }

print(f"{'Graph':<12} {'ρ(G)':<10} {'a(G)':<10} {'Energy':<10}")
print("-" * 45)
for name, G in graphs.items():
    props = spectral_properties(G)
    print(f"{name:<12} {props['spectral_radius']:<10.4f} {props['algebraic_connectivity']:<10.4f} {props['energy']:<10.4f}")

## Eigenvalue Distribution: Histogram View

For larger graphs, histograms show the eigenvalue distribution more clearly.

In [ ]:
# Generate a larger random graph for demonstration
G_random = nx.gnm_random_graph(50, 150, seed=42)

adj_eigs = np.sort(nx.adjacency_spectrum(G_random).real)
lap_eigs = np.sort(nx.laplacian_spectrum(G_random).real)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(adj_eigs, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Eigenvalue')
axes[0].set_ylabel('Count')
axes[0].set_title('Adjacency Spectrum of G(50, 150)')
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)

axes[1].hist(lap_eigs, bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_xlabel('Eigenvalue')
axes[1].set_ylabel('Count')
axes[1].set_title('Laplacian Spectrum of G(50, 150)')

plt.tight_layout()
plt.show()

print(f"Spectral radius: {np.max(np.abs(adj_eigs)):.4f}")
print(f"Algebraic connectivity: {lap_eigs[1]:.4f}")

## Comparing Graph Families

Let's compare how spectral properties scale with graph size for different families.

In [ ]:
# Compare cycles, paths, and complete graphs of various sizes
sizes = range(4, 15)

results = {'cycle': [], 'path': [], 'complete': []}

for n in sizes:
    for family, constructor in [('cycle', nx.cycle_graph), 
                                  ('path', nx.path_graph), 
                                  ('complete', nx.complete_graph)]:
        G = constructor(n)
        props = spectral_properties(G)
        results[family].append(props)

# Plot spectral radius vs n
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for family, color in [('cycle', 'blue'), ('path', 'green'), ('complete', 'red')]:
    radii = [r['spectral_radius'] for r in results[family]]
    alg_conn = [r['algebraic_connectivity'] for r in results[family]]
    energy = [r['energy'] for r in results[family]]
    
    axes[0].plot(list(sizes), radii, 'o-', label=family, color=color)
    axes[1].plot(list(sizes), alg_conn, 'o-', label=family, color=color)
    axes[2].plot(list(sizes), energy, 'o-', label=family, color=color)

axes[0].set_xlabel('n')
axes[0].set_ylabel('Spectral radius')
axes[0].set_title('Spectral Radius vs Graph Size')
axes[0].legend()

axes[1].set_xlabel('n')
axes[1].set_ylabel('Algebraic connectivity')
axes[1].set_title('Algebraic Connectivity vs Graph Size')
axes[1].legend()

axes[2].set_xlabel('n')
axes[2].set_ylabel('Energy')
axes[2].set_title('Graph Energy vs Graph Size')
axes[2].legend()

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Loading graphs from graph6 encoding using NetworkX
- Computing adjacency and Laplacian eigenvalues
- Visualizing spectra as scatter plots and histograms
- Extracting key spectral invariants (spectral radius, algebraic connectivity, energy)
- Comparing spectral properties across graph families

For more advanced analysis including non-backtracking matrices, see the SMOL database and API.